In [ ]:
import pandas as pd
import numpy as np
import csv
from fuzzywuzzy import process
import ftfy

In [ ]:
df_csr = pd.read_csv('../faculty-affiliations.csv')
csr_unis = list(df_csr.affiliation.unique())
csr_names = list(df_csr.name.unique())
csr_names_dict = {k:1 for k in csr_names}

In [ ]:
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('../dblp-aliases.csv')

In [ ]:
df_profs = pd.read_excel('all_professors.xlsx',encoding='utf-8')


In [ ]:
name_map = pd.read_csv('map_from_names.csv')

In [ ]:
df_profs_merged = pd.merge(df_profs,name_map,left_on='UniversityID',right_on='ID')
df_profs_merged.school = df_profs_merged.school.str.strip()
scr_unis = df_profs_merged.school.unique()
new_uni = [n for n in scr_unis if n not in csr_unis]
old_uni = [n for n in scr_unis if n  in csr_unis]


In [ ]:
len(csr_unis),len(scr_unis),len(new_uni),len(old_uni),new_uni

In [ ]:
for uni in new_uni:
    print(uni)#,'\t',process.extractOne(uni,csr_unis))

In [ ]:
from fuzzywuzzy import fuzz
row_alts = []
csr_names = csr_names
uni = 'Carnegie Mellon University'
csr_names = list(df_csr[df_csr.affiliation == uni].name)
print(len(csr_names),df_profs_merged[df_profs_merged.school == uni].shape[0])
for row in df_profs_merged[df_profs_merged.school == uni].itertuples():
    n = ' '.join([row[3],row[2]])
    n = aliasdict.get(n,n)
    matcher = fuzz.SequenceMatcher(None, fuzz._process_and_sort(n,False))
    n3s = []
    for n2 in csr_names:
        matcher.set_seq2(fuzz._process_and_sort(n2,False))

        n3s.append(matcher.ratio())
    v=np.argmax(n3s)
    print(fuzz._process_and_sort(n,False),csr_names[v],n3s[v])
    #three_alts = process.extractBests(n,csr_names,limit=3)
    #row_alts.append(three_alts)
    #print(n,'\t\t\t\t\t',three_alts[0],n in aliasdict)

In [ ]:
import gzip
import pickle
with gzip.open('../useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('../useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)

In [ ]:
author_counts = {}
for paper in all_papers:
    for a in paper[2]:
        author_counts[a] = 1 + author_counts.get(a,0)

In [ ]:
viable_authors = [a for a,c in author_counts.items() if c >= 10 ]
len(viable_authors),len(all_authors)

In [ ]:
matchable_names = [fuzz._process_and_sort(n,False) for n in viable_authors]

In [ ]:
matched_names = []
for row in df_profs_merged.itertuples():
    n = ' '.join([row[3],row[2]])
    n = aliasdict.get(n,n)
    matcher = fuzz.SequenceMatcher(None, fuzz._process_and_sort(n,False))
    n3s = []
    for n2 in matchable_names:
        matcher.set_seq2(n2)
        n3s.append(matcher.ratio())
    v=np.argmax(n3s)
    res = (n,viable_authors[v],n3s[v])
    matched_names.append(res)
    print(res)

In [ ]:
df_profs_merged['First Name']= df_profs_merged['First Name'].fillna('')
df_profs_merged['Last Name']= df_profs_merged['Last Name'].fillna('')

In [ ]:
import unidecode

In [ ]:
all_author_min = [unidecode.unidecode(n).lower().strip().replace('.','').replace('(','').replace(')','').replace('-','') for n in viable_authors]
xcl_author_min = [unidecode.unidecode(' '.join([row[3],row[2]])).lower().strip().replace('.','').replace('(','').replace(')','').replace('-','') for row in df_profs_merged.itertuples()]
xcl_author_max = [' '.join([row[3],row[2]]) for row in df_profs_merged.itertuples()]

all_author_min_set = {k:i for i,k in enumerate(all_author_min)}


In [ ]:
xcl_nomid = {}
xcl_nomidl = []
for n in xcl_author_min:
    ns = n.split(' ')
    if len(ns) > 2:
        n = ns[0] + ' ' + ns[-1]
    xcl_nomid[n] = 1 + xcl_nomid.get(n,0)
    xcl_nomidl.append(n)

In [ ]:
all_nomid = {}
for i,n in enumerate(all_author_min):
    ns = n.split(' ')
    if len(ns) > 2:
        nn = ns[0] + ' ' + ns[-1]
    else:
        nn = n
    on = all_nomid.get(nn,[])
    if on is None:
        on = []
    all_nomid[nn] =  on + [i]

In [ ]:
merged_names = []
exists = []
for idx, name in enumerate(xcl_nomidl):
    new_name = df_profs_merged.loc[idx,'First Name'] + ' ' + df_profs_merged.loc[idx,'Last Name']

    if df_profs_merged.loc[idx,'Full Name'].lower().strip() in all_author_min_set:
        new_name = viable_authors[all_author_min.index(df_profs_merged.loc[idx,'Full Name'].lower().strip())]
        exists.append(1)
    elif xcl_nomid[name] == 1 and name in all_nomid and len(all_nomid[name]) == 1:
        new_name = viable_authors[all_nomid[name][0]]
        exists.append(1)
    elif xcl_nomid[name] == 1 and xcl_author_min[idx] in all_author_min_set:
        new_name = viable_authors[all_author_min.index(xcl_author_min[idx])]
        exists.append(1)
    else:
        #if name in all_nomid:
        #    counts = [(viable_authors[idx],author_counts[viable_authors[idx]]) for idx in all_nomid[name]]
        #    res = process.extractWithoutOrder(xcl_author_max[idx],[viable_authors[idx] for idx in all_nomid[name]])
        #    #print(xcl_author_max[idx],counts,list(res))
        #else:
        #    full_name = xcl_author_max[idx]
        #    matcher = fuzz.SequenceMatcher(None, fuzz._process_and_sort(full_name,False))
        #    n3s = []
        #    for n2 in matchable_names:
        #        matcher.set_seq2(n2)
        #        n3s.append(matcher.ratio())
        #    v=np.argmax(n3s)
        #    res = (full_name,viable_authors[v],n3s[v])
        #    #print(res)        
        exists.append(0)

    merged_names.append(new_name)

In [ ]:
sum(exists),len(exists)

In [ ]:
'yang richard yang' in all_author_min_set

In [ ]:
matched_num2 = []
matched_names2 = []
for n,i in zip(merged_names,exists):
    print(n,i)
    n2 = n
    if i == 0:
        matcher = fuzz.SequenceMatcher(None, fuzz._process_and_sort(n,False))
        n3s = []
        for n2 in matchable_names:
            matcher.set_seq2(n2)
            n3s.append(matcher.ratio())
            #atcher.
        v=np.argmax(n3s)
        res = (n,viable_authors[v],n3s[v])
        print(res)
        n2 = res[-2]
        matched_num2.append(res[-1])
    else:
        matched_num2.append(0)
    matched_names2.append(n2)

In [ ]:
df_profs_merged[(df_profs_merged.fuzzyscore >= 0.96) & (df_profs_merged.fuzzyscore < 1)].shape

In [ ]:
df_profs_merged['dblpname'] = merged_names
df_profs_merged['dblpexists'] = exists
df_profs_merged['fuzzyscore'] = matched_num2
df_profs_merged['fuzzyname'] = matched_names2

In [ ]:
ftfy.fix_text(str(df_profs_merged.loc[50,'Full Name']))

In [ ]:
df_profs_merged.to_csv('../faculty_affil_scholar.csv')

In [ ]:
for uni in new_uni:
    for row in df_profs_merged[(df_profs_merged.school == uni) & (df_profs_merged.dblpexists == 1)].itertuples():
        print(row[-2],row[-3],'NOSCHOLARPAGE' if str(row[4]) == 'nan' else row[4][row[4].index('AAAAJ')-7:row[4].index('AAAAJ')+5] )

In [ ]:
old_names =[aliasdict.get(a,a) for a in list( df_csr.name)]
matchable_names = [fuzz._process_and_sort(n,False) for n in old_names]

for uni in old_uni:
    for row in df_profs_merged[(df_profs_merged.school == uni) & (df_profs_merged.dblpexists == 1)].itertuples():
            n = aliasdict.get(row[-2],row[-2])
            try:
                if n not in old_names:
                   # matcher = fuzz.SequenceMatcher(None, fuzz._process_and_sort(n,False))
                    #n3s = []
                    #for n2 in matchable_names:
                    #    matcher.set_seq2(n2)
                   #    n3s.append(matcher.quick_ratio())
                        #atcher.
                    #v=np.argmax(n3s)
                    #res = (n,all_authors[v],n3s[v])
                    res = process.extractOne(n,old_names)

                    if(res[-1] < 95):
                        print(n,res)
                        print(n,row[-3],'NOSCHOLARPAGE' if str(row[4]) == 'nan' else row[4][row[4].index('AAAAJ')-7:row[4].index('AAAAJ')+5] )
            except KeyboardInterrupt:
                raise
            except:
                pass

In [ ]:
df_profs_merged

In [ ]:
import re, urllib
import pandas as pd
from bs4 import BeautifulSoup
import time
import urllib
import requests

f = open("more2.csv", "at")

for uni in old_uni:
    for row in df_profs_merged[(df_profs_merged.school == uni) & (df_profs_merged.dblpexists == 1)].itertuples():
            n = aliasdict.get(row[-2],row[-2])
            try:
                if n not in old_names:
                    res = process.extractOne(n,old_names)

                    if(res[-1] < 95):
                        query = (row[-2] + ' ' + row[-3]).replace(' ','+')
                        results = requests.get("http://www.bing.com/search", 
                                      params={'q': query, 'first': 0}, 
                                      headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 Chrome/65.0.3325.162 Safari/537.36'})
                        soup = BeautifulSoup(results.text)


                        res1 = soup.findAll('li', {"class" : "b_algo" })
                        res2 = [li.find('a') for li in res1]
                        res3 = [link.get('href') for link in res2]#[0]
                        url = res3[0]
                        res4 = [link for link in res3 if 'edu' in link]
                        if len(res4) > 0:
                            url = res4[0]

                        new_entry = [row[-2],row[-3],url,'NOSCHOLARPAGE' if str(row[4]) == 'nan' else row[4][row[4].index('AAAAJ')-7:row[4].index('AAAAJ')+5]]
                        print(','.join(new_entry))
                        f.write(','.join(new_entry))
                        f.write('\n')
                        time.sleep(1)
            except KeyboardInterrupt:
                raise
            except:
                pass
f.close()


In [ ]:
import re, urllib
import pandas as pd
from bs4 import BeautifulSoup
import time
import urllib
f = open("more.csv", "at")


for uni in new_uni:
    for row in df_profs_merged[(df_profs_merged.school == uni) & (df_profs_merged.dblpexists == 1)].itertuples():
        #query = (row[-2] + ' ' + row[-3]).replace(" ", "%20")
        import requests
        query = (row[-2] + ' ' + row[-3]).replace(' ','+')
        results = requests.get("http://www.bing.com/search", 
                      params={'q': query, 'first': 0}, 
                      headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 Chrome/65.0.3325.162 Safari/537.36'})
        soup = BeautifulSoup(results.text)


        res1 = soup.findAll('li', {"class" : "b_algo" })
        res2 = [li.find('a') for li in res1]
        res3 = [link.get('href') for link in res2]#[0]
        url = res3[0]
        res4 = [link for link in res3 if 'edu' in link]
        if len(res4) > 0:
            url = res4[0]
            
        new_entry = [row[-2],row[-3],url,'NOSCHOLARPAGE' if str(row[4]) == 'nan' else row[4][row[4].index('AAAAJ')-7:row[4].index('AAAAJ')+5]]
        print(','.join(new_entry))
        f.write(','.join(new_entry))
        time.sleep(1)
f.close()

In [ ]:
f = open("more.csv", "r").read()
def grouplen(sequence, chunk_size):
    return list(zip(*[iter(sequence)] * chunk_size))

In [ ]:
f2 = open('evenmore.csv','wt')
f2.write('\n'.join([','.join(a) for a in grouplen(f.split(','),4)]))
f2.close()

In [ ]:
grouplen(f.split(','),4)

In [ ]:
soup.findAll('li', {"class" : "b_algo" })